Лабораторная работа №2

Вашинко Евгений

In [8]:
import numpy as np

In [9]:
def next_val(m, k, prev):
    return (k * prev) % m


def multiplicative_congruent_method(m, k, a0):
    next = a0
    while True:
        yield next / m
        next = next_val(m, k, next)


def get_gen():
    m = 131072
    k = 131101
    a0 = 29
    return multiplicative_congruent_method(m, k, a0)

Задание 1. Имитация простого случайного события.

In [10]:
class Event:
    def __init__(self):
        self.gen = get_gen()

    def check_event(self, p):
        val = next(self.gen)
        return val < p


def check(p):
    e = Event()
    n = 10 ** 6
    count = 0
    for i in range(n):
        if e.check_event(p):
            count = count + 1
    return count


print(check(0.1))
print(check(0.5))
print(check(0.9))


99993
500072
900011


Задание 2. Имитация сложного события

In [11]:
class Event:
    def __init__(self):
        self.gen = get_gen()

    def check_event(self, p):
        val = next(self.gen)
        return val < p

    def check_events(self, p_list):
        result = list()
        for p in p_list:
            result.append(self.check_event(p))
        return np.array(result, dtype=int)


def check(p_list):
    e = Event()
    n = 10 ** 6
    count = e.check_events(p_list)
    for i in range(n - 1):
        count += e.check_events(p_list)
    return count


print(check([0.1, 0.3, 0.6]))
print(check([0.6, 0.4]))
print(check([0.9, 0.1]))


[100040 299998 600087]
[599965 400013]
[899955 100023]


Задание 3. Имитация сложного события, состоящего из зависимых
событий.

In [15]:
class Event:
    def __init__(self):
        self.gen = get_gen()

    def check_event(self, p):
        val = next(self.gen)
        return val < p

    def check_events(self, p1, p2):
        result = np.array([0, 0, 0, 0])
        if self.check_event(p1):
            if self.check_event(p2):
                result[0] = 1
            else:
                result[1] = 1
        else:
            if not self.check_event(p2):
                result[2] = 1
            else:
                result[3] = 1
        return result


def check(p1, p2):
    e = Event()
    n = 10 ** 6
    count = e.check_events(p1, p2)
    for i in range(n - 1):
        count += e.check_events(p1, p2)
    return count


print(check(0.1, 0.3))
print(check(0.6, 0.4))
print(check(0.8, 0.2))


[ 30940  69022 630946 269092]
[248280 351685 248302 151733]
[165508 634472 165540  34480]


Задание 4. Имитация событий, образующих полную группу

In [13]:
class Event:
    def __init__(self):
        self.gen = get_gen()

    def check_event(self, p_list):
        val = next(self.gen)
        p_list = np.cumsum(p_list)
        # print(p_list)
        result = np.array([0] * len(p_list))
        for i, p in enumerate(p_list):
            if val <= p:
                result[i] = 1
                return result


def check(p_list):
    e = Event()
    n = 10 ** 6
    count = e.check_event(p_list)
    for i in range(n - 1):
        count += e.check_event(p_list)
    return count


print(check([0.1, 0.3, 0.6]))
print(check([0.6, 0.4]))
print(check([0.9, 0.1]))


[ 99993 300050 599957]
[600070 399930]
[900011  99989]


Дополнительное задание. Вариант 3. «Колесо фортуны» для стримера.

In [16]:
class Event:
    def __init__(self):
        self.gen = mcm.get_gen()

    def check_event(self, games):
        val = next(self.gen)
        keys = [key for key in games]
        values = [games[key] for key in keys]
        values = np.cumsum(values)
        for i, p in enumerate(values):
            if val <= p:
                return keys[i]

games = dict()

while True:
    s = input()
    if s == 'stop':
        break
    game, price = s.split(',')
    if game not in games:
        games[game] = int(price)
    else:
        games[game] += int(price)
print(games)

total = sum(games.values())
for key in games:
    games[key] = games[key] / total

e = Event()
game_stat = dict()
for i in range(10000):
    game = e.check_event(games)
    if game not in game_stat:
        game_stat[game] = 1
    else:
        game_stat[game] += 1

print(games)
print(game_stat)

Civilization VI, 1000
Formula 1, 400
PUBG, 500
Witcher 3, 600
Civilization VI, 275
Witcher 3, 300
stop
{'Civilization VI': 1275, 'Formula 1': 400, 'PUBG': 500, 'Witcher 3': 900}
{'Civilization VI': 0.4146341463414634, 'Formula 1': 0.13008130081300814, 'PUBG': 0.16260162601626016, 'Witcher 3': 0.2926829268292683}
{'Civilization VI': 4166, 'Formula 1': 1306, 'Witcher 3': 2956, 'PUBG': 1572}
